In [8]:
import time

import pickle
import pandas as pd

from tqdm import tqdm
import numpy as np

file cleaned_data.sav from step 1.Pre.ipynb

In [9]:
in_file  = "./cleaned_data.sav"
df = pd.read_pickle(open(in_file, 'rb'))

sav to json

In [10]:
abstract = df['abstract']
body_text = df['body_text']

In [11]:
import spacy
import json

In [12]:
def to_json(df, save_file_name):
    results = []
    nlp = spacy.load('en_core_web_lg')

    def to_sent(text):
        doc=nlp(text)
        result=[]
        for sent in list(doc.sents):
            sent=str(sent)
            sent = sent.rstrip() #remove end space
            result.append(sent)
        return result
    
    for text in tqdm(df):
        results += to_sent(text)
    
    json_file = save_file_name + '.json'
    with open(json_file,'w') as f:
        json.dump(results, f)

In [ ]:
to_json(abstract, 'abstract')

  2%|█▊                                                                        | 3889/160501 [02:00<1:25:30, 30.52it/s]

In [ ]:
to_json(body_text, 'body_text')

minie

filter by biobert

In [62]:
csv_file = "result.csv"
csv_data = pd.read_csv(csv_file,encoding='utf-8')
csv_df = pd.DataFrame(csv_data)

In [72]:
# remove QUANT_
df = csv_df.dropna()[~csv_df.dropna()['subject'].str.contains('QUANT_')]
df = df.dropna()[~df.dropna()['object'].str.contains('QUANT_')]

In [73]:
pronounsList = ["another", "anybody", "anyone", "anything", "each", "either", "enough", "everybody", "everyone",
                "everything", "little", "much", "neither", "nobody", "no one", "nothing", "one", "other", "somebody",
                "something", "both", "few", "fewer", "many", "others", "several", "all", "any", "more", "most",
                "someone", "none", "some", "such", "I", "me", "my", "mine", "you", "your", "yours", "yourself",
                "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "myself", "it", "its", 
                "itself", "we", "us", "our", "ours", "ourselves", "they", "them", "their", "theirs", "themselves"]

In [74]:
# remove single pronoun
df = df.dropna()[~df.dropna()['subject'].str.lower().isin(pronounsList)]
df = df.dropna()[~df.dropna()['object'].str.lower().isin(pronounsList)]

In [76]:
# label sentence
df['sentence'] = df['subject'].map(str) + " " + df['relation'] + " " + df['object']

In [77]:
def label(sub,obj,sent):
    subl = sub.lower()
    objl = obj.lower()
    sentl = sent.lower()
    
    sub_location = sentl.find(subl)
    if(sub_location == -1):
        return ''
    new_sub = '@' + subl.upper() + '#'
    sentl = sent[:sub_location] + new_sub + sent[sub_location+len(subl):]
    
    obj_location = sentl.find(objl)
    if(obj_location == -1):
        return ''
    new_obj = '@' + objl.upper() + '#'
    sentl = sentl[:obj_location] + new_obj + sentl[obj_location+len(objl):]
    
    return sentl


for i,r in df.iterrows():
    df.loc[i,'label'] = label(r['subject'],r['object'],r['sentence'])

KeyboardInterrupt: 

In [ ]:
df.to_csv('all.tsv')

In [ ]:
df=df.drop(labels='subject')
df=df.drop(labels='relation')
df=df.drop(labels='object')
df.insert(0, 'index', range(len(df)), allow_duplicates=False)

In [ ]:
df.columns = ['index','sentence','label']

In [ ]:
df

In [ ]:
df.to_csv('test.tsv', sep="\t" , index=False)

In [ ]:
run biobert